In [1]:
pip install torch transformers scikit-learn pandas numpy matplotlib seaborn tqdm nlpaug 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install accelerate -U

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import resample
import re
import string
from nlpaug.augmenter.word import SynonymAug
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
data_path = '../Data/labeled_data.csv'
df = pd.read_csv(data_path, encoding='latin1', delimiter=',', quotechar='"')

# Inspect the dataset
print(df.head())
print(df.info())
print(df.describe())

# Check for missing values
print(df.isnull().sum())

# Drop rows with missing values
df.dropna(subset=['tweet'], inplace=True)

# Map classes to binary labels
df['label'] = df['class'].apply(lambda x: 1 if x == 0 else 0)  # 1: Hate Speech, 0: Non-Hate Speech

# Drop unnecessary columns
df = df[['tweet', 'label']]

# Check class distribution
print(df['label'].value_counts())

C:\Users\Aditya Mohan Khade\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24

In [4]:
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove mentions (@user)
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags (#hashtag)
    text = re.sub(r'#\w+', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convert to lowercase
    text = text.lower()
    return text

df['clean_tweet'] = df['tweet'].apply(preprocess_text)

In [5]:
# Separate majority and minority classes
df_majority = df[df.label == 0]  # Non-Hate Speech
df_minority = df[df.label == 1]  # Hate Speech

# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # Sample with replacement
                                 n_samples=len(df_majority),    # Match majority class
                                 random_state=42)  # Reproducible results

# Combine majority class with upsampled minority class
df_balanced = pd.concat([df_majority, df_minority_upsampled])

# Verify balanced class distribution
print(df_balanced['label'].value_counts())

0    23353
1    23353
Name: label, dtype: int64


In [6]:
# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to tokenize and encode data
def encode_data(texts, tokenizer, max_length=128):
    encodings = tokenizer(list(texts), truncation=True, padding=True, max_length=max_length, return_tensors='pt')
    return encodings

# Encode training and testing data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_balanced['clean_tweet'], df_balanced['label'], test_size=0.2, random_state=42, stratify=df_balanced['label']
)

train_encodings = encode_data(train_texts, tokenizer)
val_encodings = encode_data(val_texts, tokenizer)

C:\Users\Aditya Mohan Khade\AppData\Roaming\Python\Python310\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
class HateSpeechDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        return item

# Create datasets
train_dataset = HateSpeechDataset(train_encodings, train_labels)
val_dataset = HateSpeechDataset(val_encodings, val_labels)

In [ ]:
# Initialize model
model_bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Define training arguments
training_args_bert = TrainingArguments(
    output_dir='./results_bert',
    evaluation_strategy='epoch',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize trainer
trainer_bert = Trainer(
    model=model_bert,
    args=training_args_bert,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer_bert.train()

# Evaluate the model
results_bert = trainer_bert.evaluate()
print(results_bert)

In [ ]:
# Predict on validation set
predictions_bert = trainer_bert.predict(val_dataset)
pred_labels_bert = predictions_bert.predictions.argmax(-1)

# Generate classification report
print(classification_report(val_labels, pred_labels_bert, target_names=['Non-Hate Speech', 'Hate Speech']))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class LSTM_CNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, lstm_hidden_dim, cnn_hidden_dim, num_classes, dropout=0.5):
        super(LSTM_CNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, lstm_hidden_dim, bidirectional=True, batch_first=True)
        self.conv = nn.Conv1d(in_channels=embed_dim, out_channels=cnn_hidden_dim, kernel_size=3)
        self.fc = nn.Linear(lstm_hidden_dim * 2 + cnn_hidden_dim, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids):
        embedded = self.embedding(input_ids)
        embedded_permuted = embedded.permute(0, 2, 1)  # Permute for CNN
        lstm_out, _ = self.lstm(embedded)
        conv_out = self.conv(embedded_permuted)
        conv_out = F.relu(conv_out)
        conv_out = F.max_pool1d(conv_out, conv_out.size(2)).squeeze(2)
        combined_out = torch.cat((lstm_out[:, -1, :], conv_out), dim=1)
        combined_out = self.dropout(combined_out)
        logits = self.fc(combined_out)
        return logits

10

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Vectorize the text data
vectorizer = CountVectorizer(max_features=10000)
X_vectorized = vectorizer.fit_transform(df_balanced['clean_tweet']).toarray()
X_train_lstm, X_val_lstm, y_train_lstm, y_val_lstm = train_test_split(
    X_vectorized, df_balanced['label'], test_size=0.2, random_state=42, stratify=df_balanced['label']
)

# Convert to tensors
X_train_tensor = torch.tensor(X_train_lstm, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_lstm.values, dtype=torch.long)
X_val_tensor = torch.tensor(X_val_lstm, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val_lstm.values, dtype=torch.long)

# Create datasets
train_dataset_lstm = Dataset.from_tensor_slices((X_train_tensor, y_train_tensor))
val_dataset_lstm = Dataset.from_tensor_slices((X_val_tensor, y_val_tensor))

In [ ]:
# Hyperparameters
vocab_size = 10000
embed_dim = 128
lstm_hidden_dim = 128
cnn_hidden_dim = 128
num_classes = 2
dropout = 0.5

# Initialize model
model_lstm_cnn = LSTM_CNN(vocab_size, embed_dim, lstm_hidden_dim, cnn_hidden_dim, num_classes, dropout)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_lstm_cnn.parameters(), lr=0.001)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_lstm_cnn.to(device)

num_epochs = 3
batch_size = 8

train_loader_lstm_cnn = DataLoader(train_dataset_lstm, batch_size=batch_size, shuffle=True)
val_loader_lstm_cnn = DataLoader(val_dataset_lstm, batch_size=batch_size)

for epoch in range(num_epochs):
    model_lstm_cnn.train()
    total_loss = 0
    for inputs, labels in train_loader_lstm_cnn:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model_lstm_cnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader_lstm_cnn)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

    # Evaluation
    model_lstm_cnn.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for inputs, labels in val_loader_lstm_cnn:
            inputs = inputs.to(device)
            outputs = model_lstm_cnn(inputs)
            preds = torch.argmax(outputs, dim=1)
            total_correct += (preds == labels).sum().item()
            total_samples += labels.size(0)
    
    accuracy = total_correct / total_samples
    print(f"Validation Accuracy: {accuracy:.4f}")

In [ ]:
from sklearn.metrics import classification_report

# Predict on validation set
model_lstm_cnn.eval()
preds_list = []
with torch.no_grad():
    for inputs, labels in val_loader_lstm_cnn:
        inputs = inputs.to(device)
        outputs = model_lstm_cnn(inputs)
        preds = torch.argmax(outputs, dim=1)
        preds_list.extend(preds.cpu().numpy())

# Generate classification report
print(classification_report(y_val_lstm, preds_list, target_names=['Non-Hate Speech', 'Hate Speech']))

In [ ]:
# Initialize synonym augmentation
aug = SynonymAug(aug_src='wordnet')

# Augment text
augmented_texts = [aug.augment(text) for text in train_texts]

# Combine original and augmented texts
train_texts_augmented = train_texts.tolist() + augmented_texts
train_labels_augmented = train_labels.tolist() + train_labels.tolist()

# Shuffle the augmented data
augmented_df = pd.DataFrame({'clean_tweet': train_texts_augmented, 'label': train_labels_augmented})
augmented_df = augmented_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Encode augmented data
train_encodings_augmented = encode_data(augmented_df['clean_tweet'], tokenizer)

# Create augmented dataset
train_dataset_augmented = HateSpeechDataset(train_encodings_augmented, augmented_df['label'])

In [ ]:
from transformers import AdamW

# Define function for adversarial training
def adversarial_training(model, inputs, labels, epsilon=0.01):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(**inputs, labels=labels)
    loss = outputs.loss
    loss.backward()
    
    # Get gradient of loss w.r.t. input embeddings
    embed_grad = inputs['input_ids'].grad
    
    # Create adversarial perturbation
    perturbed_inputs = inputs['input_ids'] + epsilon * embed_grad.sign()
    
    # Forward pass with perturbed inputs
    adv_outputs = model(input_ids=perturbed_inputs, attention_mask=inputs['attention_mask'], labels=labels)
    adv_loss = adv_outputs.loss
    
    # Backpropagate adversarial loss
    adv_loss.backward()
    optimizer.step()

# Reinitialize model and optimizer
model_adv = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
optimizer_adv = AdamW(model_adv.parameters(), lr=2e-5)

# Training loop with adversarial training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_adv.to(device)

num_epochs = 3
batch_size = 8

train_loader_adv = DataLoader(train_dataset_augmented, batch_size=batch_size, shuffle=True)
val_loader_adv = DataLoader(val_dataset, batch_size=batch_size)

for epoch in range(num_epochs):
    model_adv.train()
    total_loss = 0
    for batch in train_loader_adv:
        inputs = {key: val.to(device) for key, val in batch.items()}
        labels = inputs.pop('labels')
        
        optimizer_adv.zero_grad()
        outputs = model_adv(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward(retain_graph=True)  # Retain graph for adversarial training
        
        adversarial_training(model_adv, inputs, labels)
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader_adv)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

    # Evaluation
    model_adv.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for batch in val_loader_adv:
            inputs = {key: val.to(device) for key, val in batch.items()}
            labels = inputs.pop('labels')
            
            outputs = model_adv(**inputs)
            preds = torch.argmax(outputs.logits, dim=1)
            total_correct += (preds == labels).sum().item()
            total_samples += labels.size(0)
    
    accuracy = total_correct / total_samples
    print(f"Validation Accuracy: {accuracy:.4f}")

In [ ]:
# Predict on validation set
model_adv.eval()
preds_list_adv = []
with torch.no_grad():
    for batch in val_loader_adv:
        inputs = {key: val.to(device) for key, val in batch.items()}
        labels = inputs.pop('labels')
        
        outputs = model_adv(**inputs)
        preds = torch.argmax(outputs.logits, dim=1)
        preds_list_adv.extend(preds.cpu().numpy())

# Generate classification report
print(classification_report(val_labels, preds_list_adv, target_names=['Non-Hate Speech', 'Hate Speech']))

In [ ]:
# Plotting classification reports
def plot_classification_report(report, title):
    report_df = pd.DataFrame(report).iloc[:-1, :].T
    plt.figure(figsize=(10, 6))
    sns.heatmap(report_df, annot=True, fmt=".2f", cmap='Blues')
    plt.title(title)
    plt.show()

# Classification report for BERT
report_bert = classification_report(val_labels, pred_labels_bert, target_names=['Non-Hate Speech', 'Hate Speech'], output_dict=True)
plot_classification_report(report_bert, "BERT Model Classification Report")

# Classification report for LSTM + CNN
report_lstm_cnn = classification_report(y_val_lstm, preds_list, target_names=['Non-Hate Speech', 'Hate Speech'], output_dict=True)
plot_classification_report(report_lstm_cnn, "LSTM + CNN Model Classification Report")

# Classification report for Adversarial Training
report_adv = classification_report(val_labels, preds_list_adv, target_names=['Non-Hate Speech', 'Hate Speech'], output_dict=True)
plot_classification_report(report_adv, "Adversarial Training Model Classification Report")

In [ ]:
# Save the BERT model
model_bert.save_pretrained('./hate_speech_model_bert')
tokenizer.save_pretrained('./hate_speech_model_bert')

# Save the LSTM + CNN model
torch.save(model_lstm_cnn.state_dict(), './hate_speech_model_lstm_cnn.pth')

# Save the Adversarial Training model
model_adv.save_pretrained('./hate_speech_model_adv')
tokenizer.save_pretrained('./hate_speech_model_adv')

print("Models saved successfully!")

In [ ]:
from flask import Flask, request, jsonify
from transformers import pipeline

app = Flask(__name__)

# Load the trained BERT model
classifier_bert = pipeline('text-classification', model='./hate_speech_model_bert')

@app.route('/predict_bert', methods=['POST'])
def predict_bert():
    data = request.json
    text = data['text']
    result = classifier_bert(text)
    return jsonify(result)

# Load the trained LSTM + CNN model
class LSTM_CNN_Deploy(nn.Module):
    def __init__(self, vocab_size, embed_dim, lstm_hidden_dim, cnn_hidden_dim, num_classes, dropout=0.5):
        super(LSTM_CNN_Deploy, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, lstm_hidden_dim, bidirectional=True, batch_first=True)
        self.conv = nn.Conv1d(in_channels=embed_dim, out_channels=cnn_hidden_dim, kernel_size=3)
        self.fc = nn.Linear(lstm_hidden_dim * 2 + cnn_hidden_dim, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids):
        embedded = self.embedding(input_ids)
        embedded_permuted = embedded.permute(0, 2, 1)  # Permute for CNN
        lstm_out, _ = self.lstm(embedded)
        conv_out = self.conv(embedded_permuted)
        conv_out = F.relu(conv_out)
        conv_out = F.max_pool1d(conv_out, conv_out.size(2)).squeeze(2)
        combined_out = torch.cat((lstm_out[:, -1, :], conv_out), dim=1)
        combined_out = self.dropout(combined_out)
        logits = self.fc(combined_out)
        return logits

# Initialize deployed LSTM + CNN model
model_deploy = LSTM_CNN_Deploy(vocab_size, embed_dim, lstm_hidden_dim, cnn_hidden_dim, num_classes, dropout)
model_deploy.load_state_dict(torch.load('./hate_speech_model_lstm_cnn.pth'))
model_deploy.to(device)
model_deploy.eval()

# Function to preprocess and encode text for LSTM + CNN
def preprocess_and_encode_lstm_cnn(text, vectorizer, tokenizer, max_length=128):
    text = preprocess_text(text)
    vectorized_text = vectorizer.transform([text]).toarray()
    vectorized_tensor = torch.tensor(vectorized_text, dtype=torch.float32).to(device)
    return vectorized_tensor

@app.route('/predict_lstm_cnn', methods=['POST'])
def predict_lstm_cnn():
    data = request.json
    text = data['text']
    inputs = preprocess_and_encode_lstm_cnn(text, vectorizer, tokenizer)
    outputs = model_deploy(inputs)
    pred_label = torch.argmax(outputs, dim=1).item()
    result = [{'label': ['Non-Hate Speech', 'Hate Speech'][pred_label], 'score': F.softmax(outputs, dim=1)[0][pred_label].item()}]
    return jsonify(result)

# Load the trained Adversarial Training model
classifier_adv = pipeline('text-classification', model='./hate_speech_model_adv')

@app.route('/predict_adv', methods=['POST'])
def predict_adv():
    data = request.json
    text = data['text']
    result = classifier_adv(text)
    return jsonify(result)

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
import requests

# Sample text
sample_text = {"text": "I hate that bitch"}

# Test BERT model
response_bert = requests.post('http://127.0.0.1:5000/predict_bert', json=sample_text)
print(response_bert.json())

# Test LSTM + CNN model
response_lstm_cnn = requests.post('http://127.0.0.1:5000/predict_lstm_cnn', json=sample_text)
print(response_lstm_cnn.json())

# Test Adversarial Training model
response_adv = requests.post('http://127.0.0.1:5000/predict_adv', json=sample_text)
print(response_adv.json())